# Getting Started

This guide covers the steps to get started with **azcausal**. 
We have split this introduction into several parts:

- Part I : **The Panel**
- Part II: **The Treatment Effect**
- Part III: **Analysis & Visualization**

### Part I: Causal Panel

The very first step before doing any causal inference is get loading the data and tranforming them in the right format. Most estimators in this framework require the data to be in form of a `Panel` object. 

Assuming we have the following data frame representing the `CaliforniaProp99` information:

In [ ]:
from azcausal.data import CaliforniaProp99

df = CaliforniaProp99().df()

df

In this example, the columns represent the following:

+ The **units**: are given by each state (`State`)
+ The **time**: is represented in years (`Year`)
+ The **outcome** is the number of packs sold (`PacksPerCapita`)
+ The **intervention** is indicated by the binary treatment column (`treated`)

We define a `Panel` as a data frame where the index represents `time` and each column a `unit`. We can extract the `outcome` from the data frame by:

In [ ]:
from azcausal.util import to_panel

outcome = to_panel(df, "Year", "State", "PacksPerCapita")

outcome.head(3)

To check whether the panel data are in fact balanced (we have an entry during each time step for each unit), we can check for `NaN` values in the data frame:

In [ ]:
print("Is Balanced:", (~outcome.isna()).all(axis=None))

Let us simulated some data would be missing

In [ ]:
not_balanced = to_panel(df.head(80), "Year", "State", "PacksPerCapita")
print("Is Balanced:", (~not_balanced.isna()).all(axis=None))

not_balanced.head(3)

Instead of extacting only one value at a time, we can also extract multiple directly by 

In [ ]:
from azcausal.util import to_panels

data = to_panels(df, "Year", "State", ["PacksPerCapita", "treated"])

data.keys()

The reason why we have introduced an object called `Panel` is to combine multiple data frames into one and to have convinient access to information about the time pre and post experiment, as well as control and treatment units. A Panel can be created by passing the *outcome* and *intervention* directly as `pd.DataFrame`:

In [ ]:
from azcausal.core.panel import CausalPanel

panel = CausalPanel(data=data).setup(outcome='PacksPerCapita', intervention='treated')

print(panel.summary())

The panel allows accessing `outcome` and `intervention` directly trough properties:

In [ ]:
panel.outcome.head(3)

or using the index function by

In [ ]:
panel['outcome'].head(3)

In [ ]:
panel.intervention.tail(3)

Moreover, the method also allows to use the most common `pandas` functions on ALL DATA at once:

In [ ]:
new_panel = panel.iloc[:, :3]

new_panel.outcome.head(3)

Also, we can use the `get` method with key word arguments. 

* if `contr == True` then only control units are returned.
* if `treat == True` then units which have been treated at least once are returned.
* if `pre == True` then time steps where no unit is treated is returned.
* if `post == True` then the time steps where at least one unit is treated.

, for example:

In [ ]:
panel.filter(target='outcome', post=True, treat=True).head(3)

For more methods please check the `Panel` immplementation directly.

Using the `CausalPanel` it is also relatively easy to plot the average control versus treatment by:

In [ ]:
import seaborn as sns
sns.set(rc={'figure.figsize':(12,4)})

import matplotlib.pyplot as plt

avg_control = panel.filter(target='outcome', contr=True).mean(axis=1).to_frame('C')
avg_treat = panel.filter(target='outcome', treat=True).mean(axis=1).to_frame('T')

plt.subplots(1, 1, figsize=(12, 4))
sns.lineplot(avg_control.join(avg_treat))
plt.axvline(panel.filter(target="intervention", pre=True).index.max(), color='black', label='intervention')

### Part II: The Treatment Effect

After bringing the data into the right format, we can use an `Estimator` to make predictions of the treatment effect. 

Like commonly done for time series it is always a good idea to quickly spot check the time series:

In [ ]:
print(panel.summary())

panel.outcome.mean(axis=1).plot(figsize=(12, 4))

For example, let us use the popular `DID` estimator to estimate the *Average Treatment Effect on the Treated (ATT)* for the panel data.

In [ ]:
from azcausal.estimators.panel.did import DID

# initialize an estimator object
estimator = DID()

# estimate the treatment effect
result = estimator.fit(panel)

# print the treatment effect summary
print(result.summary(percentage=True, cumulative=True))

Now, the treatment effect without any confidence intervals is often not that helpful. Some estimators come with error estimates out of the box and will directly provide them (e.g. `DIDRegression`). For others, we can use an `Error` estimator to attach an error and calculate confidence intervals along with it. 

The following error estimators are available:

- **Bootstrap**: Randomly sample units (with replacement) from the panel data and estimate the effect.
- **Placebo**: Only sample from control units as use them as placebo.
- **JackKnife**: Leave one out crossvalidation but removing one unit at a time.

Each error estimate requires a treatment estimated on a new panel derived from the original data set by the corresponding method.

Each estimator has a method called `estimator.error(result, method)` which takes the original `Result` object returned before and the `Error` estimation method that should be used.

In [ ]:
from azcausal.core.error import Bootstrap

se, runs = estimator.error(result, Bootstrap(n_samples=500))

print(result.summary(conf=90))

Similarly, we can use `SDID` as an estimator

In [ ]:
from azcausal.estimators.panel.sdid import SDID
from azcausal.core.error import JackKnife

# initialize an estimator object
estimator = SDID()

# estimate the treatment effect
result = estimator.fit(panel)

# here we use JackKnife which is optmized to be run with SDID
estimator.error(result, JackKnife())

# print the treatment effect summary
print(result.summary(percentage=True, cumulative=True))

### Part III: Analysis & Visualization

Lastly, we want to give some idea on how to visualize results. 

To learn more about how the error estimation was originally derived, we can check the corresponding distribution by plotting the estimates of each of the runs:

In [ ]:
vv = [run.effect.value for run in runs]
sns.histplot(vv, kde=True)

Each post-analysis will be different depending on the estimator. We would like a give an example of `SDID` here. The available additional information stored by the estimator are:

In [ ]:
effect = result.effect

effect.data.keys()

The DID results give us additional information about how the ATT is actually calculated:

In [ ]:
effect['did']

The unit weights (`omega`) with at least 1% contribution

In [ ]:
effect['omega'].sort_values(ascending=False).loc[lambda x: x >= 0.01]

Similarly, the time weights

In [ ]:
effect['lambd'].sort_values(ascending=False).loc[lambda x: x >= 0.01]

Or in general the treatment effect over time:

In [ ]:
# Control (C), Treatment (T), Time Weights (lambd), Intervention (W), Average Treatment Effect on the Treated (att), Counter Factual (CF)
effect.by_time.tail(5)

In [ ]:
effect.by_time[['C', 'CF', 'T']].head(3)

Some estimators will have directly a plotting method for the result:

In [ ]:
estimator.plot(result, show=False, CF=True)
None